## sentdex - 딥러닝 Pt.5

<h3> 목표 : 텐서보드 기록을 이용해서 모델 최적화하기</h3>
<hr>

In [1]:
import warnings
warnings.filterwarnings(action='ignore')

import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
import pickle
import time
from tensorflow.keras.callbacks import TensorBoard

In [2]:
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction = 0.6)
sess = tf.Session(config = tf.ConfigProto(gpu_options = gpu_options))

from tensorflow.keras.callbacks import TensorBoard

NAME = "Cats-vs-dog-cnn-64x2-{}".format(int(time.time()))
tensorboard = TensorBoard(log_dir = "logs\\{}".format(NAME))

In [ ]:
X = pickle.load(open("X.pickle", "rb"))
y = pickle.load(open("Y.pickle", "rb"))

X = X/255.0

dense_layers = [0, 1, 2]
layer_sizes = [32, 64, 128]
conv_layers = [1, 2, 3]

for dense_layer in dense_layers:
    for layer_size in layer_sizes:
        for conv_layer in conv_layers:
            NAME = "{}-conv-{}-nodes-{}-dense-{}".format(conv_layer, layer_size, dense_layer, int(time.time()))
            tensorboard = TensorBoard(log_dir = "logs\\{}".format(NAME))
            print(NAME)
            model = Sequential()

            ## Layer 1
            model.add(   Conv2D(64, (3,3), input_shape = X.shape[1:])   )  # 64개 유닛, 3x3 윈도우사이즈
            model.add(Activation("relu"))
            model.add(MaxPooling2D(pool_size = (2, 2)))
            
            for l in range(conv_layer - 1):
                ## Layer 1
                model.add(   Conv2D(64, (3,3), input_shape = X.shape[1:])   )  # 64개 유닛, 3x3 윈도우사이즈
                model.add(Activation("relu"))
                model.add(MaxPooling2D(pool_size = (2, 2)))
            
            model.add(Flatten())
            for l in range(dense_layer):
                model.add(Dense(layer_size))
                model.add(Activation("relu"))

                
            model.add(Dense(1))  # output layer
            model.add(Activation("sigmoid"))

            model.compile(loss = "binary_crossentropy",
                         optimizer = "adam",
                         metrics = ['accuracy'])

            model.fit(X, y, batch_size = 32, epochs = 5,
                      validation_split = 0.3, callbacks = [tensorboard])  # batch_size

1-conv-32-nodes-0-dense-1613144466
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 17462 samples, validate on 7484 samples


**실제론 로그 기록 (val_acc나 loss)를 보고 해야하지만 일단 이렇게 바꾼 다음:**    
    **-dense_layers = [2]**   
    **-layer_sizes = [64]**   
    **-conv_layers = [3]**   
**위의 코드를 다시 돌리고 모델을 저장하자.**

In [ ]:
model.save("64x3-CNN.model")